Load environment variables

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

Create project client and AI Search Client

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import ConnectionType
from azure.search.documents import SearchClient
import os


project = AIProjectClient.from_connection_string(
    conn_str=os.environ["AIPROJECT_CONNECTION_STRING"], credential=DefaultAzureCredential()
)

search_connection = project.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH, include_credentials=False
)

Now we create the agent with the AI Search tool so it can search it's own data

In [ ]:
from azure.ai.projects.models import AzureAISearchTool


conn_id =  f"/subscriptions/{project.scope['subscription_id']}/resourceGroups/{project.scope['resource_group_name']}/providers/Microsoft.MachineLearningServices/workspaces/{project.scope['project_name']}/connections/{search_connection.name}"

ai_search = AzureAISearchTool(index_connection_id=search_connection.id, index_name="hotel-vector")

Create the Agent

In [18]:
instructions = """
You are a helpful hotel assistant and you provide information only based on the data you have.  
If you don't have the information on your data you don't make up answer and ask more details saying
you don't have the information
"""

agent = project.agents.create_agent(
    model="gpt-4o-mini",
    name="hotel-assistant",
    instructions=instructions,
    tools=ai_search.definitions,
    tool_resources = ai_search.resources,
)
print(f"Created agent, ID: {agent.id}")





Created agent, ID: asst_uh4u5rfO9PurfeJODBdVWCKv


In [19]:
from azure.ai.projects.models import MessageRole

thread = project.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

message = project.agents.create_message(
    thread_id=thread.id,
    role=MessageRole.USER,
    content="Which hotel have beach?",
)
print(f"Created message, message ID: {message.id}")


Created thread, thread ID: thread_90iiy5veTHrGUCgnUE5JDm1Q
Created message, message ID: msg_JhSkKy7MWh9abM3sWRKDlbAu


In [20]:
run = project.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")
print(run.last_error)

Run finished with status: RunStatus.FAILED
{'code': 'rate_limit_exceeded', 'message': 'Rate limit is exceeded. Try again in 86400 seconds.'}
